### Word In Visual Genome Process

In [1]:
#!pip install --upgrade pip

In [2]:
#!pip install --upgrade regex

In [3]:
#!pip install --upgrade pandas

In [4]:
#!pip install --upgrade yellowbrick

In [5]:
#!pip install --upgrade scipy

In [6]:
#import re
#print(re.__version__)

In [7]:
#import pandas
#print(pandas.__version__)

In [8]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [9]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [10]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/4-Word In Visual Genome"

Path(path).mkdir(parents=True, exist_ok=True)

In [29]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
            df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
            df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num) 
               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [30]:
def word_group_dataframe2(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

#### Visual Genome Data

In [13]:
df_genome_objects = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Objects_Analysis.csv")
df_genome_objects.drop("image_url", inplace=True, axis=1)
df_genome_objects

,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,1058549,trees,1,tree.n.01,NaN,557,799,0,0
1,1058534,sidewalk,1,sidewalk.n.01,5046,290,722,78,308
2,1058508,building,1,building.n.01,NaN,538,222,1,0
3,1058539,street,1,street.n.01,3798578,258,359,439,283
4,1058543,wall,1,wall.n.01,NaN,535,135,0,1
...,...,...,...,...,...,...,...,...,...
2516934,3506171,number,2417997,numeral.n.01,NaN,24,28,188,228
2516935,3786799,plate,2417997,plate.n.04,NaN,54,89,238,233
2516936,3103226,road,2417997,road.n.01,NaN,83,90,312,247
2516937,3094589,road,2417997,road.n.01,NaN,121,495,2,208


In [14]:
df_genome_objects["names"].nunique()

82825

In [15]:
df_genome_attributes = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Attributes_Analysis.csv")
df_genome_attributes

,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,1058498,clock,1,"green , tall",clock.n.01,339,79,421,91
1,5046,street,1,sidewalk,street.n.01,262,714,77,328
2,5045,shade,1,NaN,shade.n.01,192,274,119,338
3,1058529,man,1,NaN,man.n.01,262,60,238,249
4,5048,sneakers,1,grey,gym_shoe.n.01,26,52,243,489
...,...,...,...,...,...,...,...,...,...
3802369,3422177,sky,2417997,"crystal clear , blue",sky.n.01,170,497,1,2
3802370,3103226,road,2417997,NaN,road.n.01,83,90,312,247
3802371,2712437,doors,2417997,NaN,door.n.01,144,50,98,127
3802372,3140197,bus,2417997,green,bus.n.01,155,225,29,109


In [16]:
df_genome_relationships = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Relationships_Analysis.csv")
df_genome_relationships

,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,1,sidewalk,1058534,sidewalk.n.01,5046,290,722,78,308,15927,along.r.01,shade,5045,shade.n.01,192,274,119,338,on
1,1,shoes,1058525,shoe.n.01,5048,28,48,388,485,15928,wear.v.01,man,1058529,man.n.01,262,60,238,249,wears
2,1,NaN,5050,headlight.n.01,NaN,15,23,514,366,15929,have.v.01,car,5049,car.n.01,98,74,479,315,has
3,1,NaN,1058508,building.n.01,NaN,536,218,1,2,15930,along.r.01,sign,1058507,sign.n.02,182,88,118,13,on
4,1,NaN,1058534,sidewalk.n.01,NaN,266,722,77,331,15931,along.r.01,tree trunk,5055,trunk.n.01,327,87,622,234,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316099,2417997,NaN,3103225,bus.n.01,NaN,193,289,14,110,4245033,along.r.01,plate,3786799,plate.n.04,54,89,238,233,on
2316100,2417997,NaN,3786800,desert.n.01,NaN,119,157,301,211,4245034,in.r.01,NaN,3103225,bus.n.01,182,287,14,110,in
2316101,2417997,NaN,3103226,road.n.01,NaN,83,90,312,247,3722727,along.r.01,bus,3103225,bus.n.01,193,289,14,110,on
2316102,2417997,NaN,2712437,door.n.01,NaN,144,50,98,127,3880229,NaN,bus,3103225,bus.n.01,193,289,14,110,with


#### Word Search In Genome Data

In [ ]:
#df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/2-Word Select Process/{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
#df_word_select

In [ ]:
#word_list = ['an','anlama','doğum','gidiyor','günün','istediğim','istediğin','istediğiniz','kutlu','olmuş','soru','söylemek','zamanki','zor','şuna']

In [ ]:
#df_word_raw = df_word_select[df_word_select["word"].isin(word_list)]
#df_word_raw.reset_index(drop=True, inplace=True)
#df_word_raw.drop(["frequency","stem"], inplace=True, axis=1)
#df_word_raw

In [18]:
df_word_raw = pd.read_excel("Turkish_45000_Process.xlsx")
df_word_raw

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
0,NUM,NaN,bir,bir,bir,a,a,18835735
1,PRON,NaN,bu,bu,bu,this,this,11062659
2,PRON,Q,ne,ne,ne,what,what,8025880
3,CCONJ,NaN,ve,ve,ve,and,and,7766036
4,ADP,NaN,için,için,için,for,for,5484109
...,...,...,...,...,...,...,...,...
995,ADJ,NaN,resmi,resmi,resmi,formal,formal,68287
996,VERB,NaN,veriyor,ver,ver,giving,give,68163
997,NOUN,NaN,okul,okul,oku,school,school,68160
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [19]:
df_word_raw_lemma_trans_list = list(set(df_word_raw["lemma_en_translate"]))
df_word_raw_word_trans_list = list(set(df_word_raw["word_en_translate"]))

In [20]:
len(df_word_raw_lemma_trans_list)

486

In [21]:
len(df_word_raw_word_trans_list)

818

In [23]:
#df_object_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 20)
df_object_search_word_result = word_group_dataframe2(df_genome_objects, df_word_raw_word_trans_list, "names")
df_object_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_object_search_word_result

,word_en_translate,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,or?,4503769,o,2415611,NaN,NaN,83,47,185,68
1,a,4441757,a,1592392,NaN,4273932,290,164,389,390
2,a,4264319,a,1592184,NaN,"4264318 , 4264300 , 4524930",184,83,507,430
3,a,4290990,a,1592657,NaN,"4290991 , 4290992 , 4290995 , 4290996 , 429100...",64,191,195,556
4,a,4265089,a,1592204,NaN,4265135,322,80,722,104
...,...,...,...,...,...,...,...,...,...,...
372461,on,1533697,calm tidal wave on ocean,51,"ocean.n.01 , wave.n.01",NaN,56,295,20,391
372462,calm,1533704,calm tidal wave on ocean,51,"ocean.n.01 , wave.n.01",NaN,36,160,328,0
372463,empty,2663164,empty toliet-paper-holder,2350477,NaN,NaN,35,50,13,275
372464,information,3965062,identification information,2327896,NaN,NaN,10,10,126,253


In [28]:
df_object_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 10)
df_object_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_object_search_word_result

,word_en_translate,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,or?,641670,o,2364350,oxygen.n.01,NaN,91,34,254,261
1,a,4269189,a,2401794,NaN,NaN,139,221,279,31
2,a,4269187,a,2401794,NaN,NaN,12,67,194,255
3,a,4269180,a,2401794,NaN,NaN,334,112,108,0
4,a,4502644,a,2389123,NaN,"4502660 , 4502650 , 4502653 , 4502654 , 4502657",256,158,150,20
...,...,...,...,...,...,...,...,...,...,...
4507071,a,2953418,chocolate/white-frosting donut,2415437,doughnut.n.02,NaN,78,135,341,149
4507072,very,3581218,noteverypictureisofthelettuce!,2352900,NaN,NaN,70,79,276,263
4507073,he,3581218,noteverypictureisofthelettuce!,2352900,NaN,NaN,70,79,276,263
4507074,on,2953418,chocolate/white-frosting donut,2415437,doughnut.n.02,NaN,78,135,341,149


In [ ]:
df_object_search_word_result["word_en_translate"].nunique()

In [ ]:
df_object_search_word_result_merge = pd.merge(df_object_search_word_result,df_word_raw,how="left",on="word_en_translate")
df_object_search_word_result_merge

In [ ]:
df_word_count = df_object_search_word_result_merge.groupby(["word_en_translate"])[["names"]].count().reset_index()
df_word_count.rename(columns={"names":"names_count"}, inplace=True)
df_word_count.sort_values(by="names_count",ascending=False, inplace=True)
df_word_count.reset_index(drop=True, inplace=True)
df_word_count

In [ ]:
df_object_search_word_result_merge.to_excel("1000_Words_Word_Object_Names.xlsx", index=False)

In [ ]:
#df_object_search_lemma_result = word_group_dataframe(df_genome_objects, df_word_raw_lemma_trans_list, "names", 10)
df_object_search_lemma_result = word_group_dataframe2(df_genome_objects, df_word_raw_lemma_trans_list, "names")
df_object_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_object_search_lemma_result

In [ ]:
df_object_search_lemma_result["lemma_en_translate"].nunique()

In [ ]:
df_object_search_lemma_result_merge = pd.merge(df_object_search_lemma_result,df_word_raw,how="left",on="lemma_en_translate")
df_object_search_lemma_result_merge

In [ ]:
df_lemma_count = df_object_search_lemma_result_merge.groupby(["lemma_en_translate"])[["image_id"]].count().reset_index()
df_lemma_count.rename(columns={"image_id":"image_id_count"}, inplace=True)
df_lemma_count.sort_values(by="image_id_count",ascending=False, inplace=True)
df_lemma_count.reset_index(drop=True, inplace=True)
df_lemma_count

In [ ]:
df_object_search_lemma_result_merge.to_excel("1000_Words_Lemma_Object_Names.xlsx", index=False)

In [ ]:
df_lemma_count.to_excel("1000_Words_Lemma_Object_Names_Count.xlsx", index=False)

In [ ]:

lemma_count_list = list(set(df_lemma_count["lemma_en_translate"]))

In [ ]:
len(lemma_count_list)

In [ ]:
df_word_raw

In [ ]:
df_lemma_not_cover = df_word_raw[~df_word_raw["lemma_en_translate"].isin(lemma_count_list)]
df_lemma_not_cover

In [ ]:
lemma_not_cover_set = set(df_lemma_not_cover["word"])
lemma_not_cover_set

In [ ]:
#df_object_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 20)
df_object_search_word_result = word_group_dataframe2(df_genome_objects, df_word_raw_word_trans_list, "names")
df_object_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_object_search_word_result

In [ ]:
df_object_search_word_result["word_en_translate"].nunique()

In [ ]:
df_object_search_word_result_merge = pd.merge(df_object_search_word_result,df_word_raw,how="left",on="word_en_translate")
df_object_search_word_result_merge

In [ ]:
df_word_count = df_object_search_word_result_merge.groupby(["word_en_translate"])[["names"]].count().reset_index()
df_word_count.rename(columns={"names":"names_count"}, inplace=True)
df_word_count.sort_values(by="names_count",ascending=False, inplace=True)
df_word_count.reset_index(drop=True, inplace=True)
df_word_count

In [ ]:
df_object_search_word_result_merge.to_excel("1000_Words_Word_Object_Names.xlsx", index=False)

In [ ]:
df_word_count.to_excel("1000_Words_Word_Object_Names_Count.xlsx", index=False)

In [ ]:
word_count_set = list(set(df_word_count["word_en_translate"]))

In [ ]:
len(word_count_set)

In [ ]:
df_word_cover = df_word_raw[df_word_raw["word_en_translate"].isin(word_count_set)]
df_word_cover

In [ ]:
word_cover_set = set(df_word_cover["word"])
word_cover_set

In [ ]:
word_cover_set.intersection(lemma_not_cover_set)

In [ ]:
len(word_cover_set.intersection(lemma_not_cover_set))

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass